In [1]:
import tensorflow as tf
import numpy as np
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32).reshape(-1, 28*28) / 255.0
x_test = x_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
x_valid, x_train = x_train[:5000], x_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [2]:
def shuffle_batch(x, y, batch_size):
    rnd_idx = np.random.permutation(len(x))
    n_batches = len(x) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        x_batch, y_batch = x[batch_idx], y[batch_idx]
        yield x_batch, y_batch

In [3]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(0)
np.random.seed(0)

In [4]:
lr = 0.01
epoch = 20
batch_size = 200
threshold = 1.0

In [5]:
n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

x = tf.placeholder(tf.float32, shape=(None, n_inputs), name="x")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(x, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(entropy, name="loss")

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [6]:
opt = tf.train.GradientDescentOptimizer(lr)
grads_and_vars = opt.compute_gradients(loss)
clipped_grads_and_vars = [(tf.clip_by_value(grad, -threshold, threshold), var)
              for grad, var in grads_and_vars]
train = opt.apply_gradients(clipped_grads_and_vars)

In [7]:
grads_and_vars

[(<tf.Tensor 'gradients/dnn/hidden1/MatMul_grad/tuple/control_dependency_1:0' shape=(784, 300) dtype=float32>,
  <tf.Variable 'hidden1/kernel:0' shape=(784, 300) dtype=float32_ref>),
 (<tf.Tensor 'gradients/dnn/hidden1/BiasAdd_grad/tuple/control_dependency_1:0' shape=(300,) dtype=float32>,
  <tf.Variable 'hidden1/bias:0' shape=(300,) dtype=float32_ref>),
 (<tf.Tensor 'gradients/dnn/hidden2/MatMul_grad/tuple/control_dependency_1:0' shape=(300, 50) dtype=float32>,
  <tf.Variable 'hidden2/kernel:0' shape=(300, 50) dtype=float32_ref>),
 (<tf.Tensor 'gradients/dnn/hidden2/BiasAdd_grad/tuple/control_dependency_1:0' shape=(50,) dtype=float32>,
  <tf.Variable 'hidden2/bias:0' shape=(50,) dtype=float32_ref>),
 (<tf.Tensor 'gradients/dnn/hidden3/MatMul_grad/tuple/control_dependency_1:0' shape=(50, 50) dtype=float32>,
  <tf.Variable 'hidden3/kernel:0' shape=(50, 50) dtype=float32_ref>),
 (<tf.Tensor 'gradients/dnn/hidden3/BiasAdd_grad/tuple/control_dependency_1:0' shape=(50,) dtype=float32>,
  <t

In [8]:
clipped_grads_and_vars

[(<tf.Tensor 'clip_by_value:0' shape=(784, 300) dtype=float32>,
  <tf.Variable 'hidden1/kernel:0' shape=(784, 300) dtype=float32_ref>),
 (<tf.Tensor 'clip_by_value_1:0' shape=(300,) dtype=float32>,
  <tf.Variable 'hidden1/bias:0' shape=(300,) dtype=float32_ref>),
 (<tf.Tensor 'clip_by_value_2:0' shape=(300, 50) dtype=float32>,
  <tf.Variable 'hidden2/kernel:0' shape=(300, 50) dtype=float32_ref>),
 (<tf.Tensor 'clip_by_value_3:0' shape=(50,) dtype=float32>,
  <tf.Variable 'hidden2/bias:0' shape=(50,) dtype=float32_ref>),
 (<tf.Tensor 'clip_by_value_4:0' shape=(50, 50) dtype=float32>,
  <tf.Variable 'hidden3/kernel:0' shape=(50, 50) dtype=float32_ref>),
 (<tf.Tensor 'clip_by_value_5:0' shape=(50,) dtype=float32>,
  <tf.Variable 'hidden3/bias:0' shape=(50,) dtype=float32_ref>),
 (<tf.Tensor 'clip_by_value_6:0' shape=(50, 50) dtype=float32>,
  <tf.Variable 'hidden4/kernel:0' shape=(50, 50) dtype=float32_ref>),
 (<tf.Tensor 'clip_by_value_7:0' shape=(50,) dtype=float32>,
  <tf.Variable 'hid

In [9]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [10]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [11]:
with tf.Session() as sess:
    init.run()
    for i in range(epoch):
        for x_batch, y_batch in shuffle_batch(x_train, y_train, batch_size):
            sess.run(train, feed_dict={x: x_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={x: x_valid, y: y_valid})
        print(i, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./save/my_model_final.ckpt")

0 Validation accuracy: 0.5998
1 Validation accuracy: 0.8308
2 Validation accuracy: 0.8762
3 Validation accuracy: 0.908
4 Validation accuracy: 0.9144
5 Validation accuracy: 0.9252
6 Validation accuracy: 0.929
7 Validation accuracy: 0.9346
8 Validation accuracy: 0.9394
9 Validation accuracy: 0.9448
10 Validation accuracy: 0.9446
11 Validation accuracy: 0.9488
12 Validation accuracy: 0.9498
13 Validation accuracy: 0.9504
14 Validation accuracy: 0.955
15 Validation accuracy: 0.9544
16 Validation accuracy: 0.9562
17 Validation accuracy: 0.9578
18 Validation accuracy: 0.9598
19 Validation accuracy: 0.9602


In [12]:
print(save_path)
print(type(save_path))

./save/my_model_final.ckpt
<class 'str'>


In [13]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
tf.set_random_seed(0)
np.random.seed(0)

In [14]:
lr = 0.01
epoch = 20
batch_size = 200
threshold = 1.0

In [15]:
# First you need to load the graph's structure. 
# The import_meta_graph() function does just that, loading the graph's operations into the default graph, 
# and returning a Saver that you can then use to restore the model's state. 
# saver = tf.train.import_meta_graph("./save/my_model_final.ckpt.meta")

In [16]:
n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

x = tf.placeholder(tf.float32, shape=(None, n_inputs), name="x")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(x, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(entropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

opt = tf.train.GradientDescentOptimizer(lr)
grads_and_vars = opt.compute_gradients(loss)
clipped_grads_and_vars = [(tf.clip_by_value(grad, -threshold, threshold), var)
                          for grad, var in grads_and_vars]
train = opt.apply_gradients(clipped_grads_and_vars)

saver = tf.train.Saver()

In [17]:
with tf.Session() as sess:
    saver.restore(sess, "./save/my_model_final.ckpt")

    for i in range(epoch):
        for x_batch, y_batch in shuffle_batch(x_train, y_train, batch_size):
            sess.run(train, feed_dict={x: x_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={x: x_valid, y: y_valid})
        print(i, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./save/my_new_model_final.ckpt")  

INFO:tensorflow:Restoring parameters from ./save/my_model_final.ckpt
0 Validation accuracy: 0.9626
1 Validation accuracy: 0.9626
2 Validation accuracy: 0.9636
3 Validation accuracy: 0.9632
4 Validation accuracy: 0.9644
5 Validation accuracy: 0.965
6 Validation accuracy: 0.9652
7 Validation accuracy: 0.9672
8 Validation accuracy: 0.9684
9 Validation accuracy: 0.9662
10 Validation accuracy: 0.9682
11 Validation accuracy: 0.968
12 Validation accuracy: 0.969
13 Validation accuracy: 0.9698
14 Validation accuracy: 0.9698
15 Validation accuracy: 0.9694
16 Validation accuracy: 0.9704
17 Validation accuracy: 0.9724
18 Validation accuracy: 0.9708
19 Validation accuracy: 0.9716
